In [1]:
import statistics as stats
import sys
import gc

In [2]:
# Allows me to import my modules
sys.path.append('./modules')
from audio_utils import *

Attempting to read settings file...
	Read successfully!


In [3]:
#### CROP CODE
file_i = 0

In [4]:
##While True
# Load in raw (unedited) audio samples
samples = read_data(data_type="raw", file_index=file_i, batch_size=10, verbose=True)
#if samples is False:
#    break
file_i += 1

no_of_samples = len(samples)
data_info = SETTINGS.data["cropped"]

Attempting to read10raw data and labels (from index0)...
	labels: {'kit_label': ['bass_drum'], 'hit_label': ['beater'], 'tech_label': ['normal']},
	augs: {}
	labels: {'kit_label': ['crash'], 'hit_label': ['stick'], 'tech_label': ['normal']},
	augs: {}
	labels: {'kit_label': ['high_tom'], 'hit_label': ['stick'], 'tech_label': ['normal']},
	augs: {}
	labels: {'kit_label': ['hi_hat'], 'hit_label': ['stick'], 'tech_label': ['normal']},
	augs: {}
	labels: {'kit_label': ['hi_hat'], 'hit_label': ['stick'], 'tech_label': ['open']},
	augs: {}
	labels: {'kit_label': ['low_tom'], 'hit_label': ['stick'], 'tech_label': ['normal']},
	augs: {}
	labels: {'kit_label': ['mid_tom'], 'hit_label': ['stick'], 'tech_label': ['normal']},
	augs: {}
	labels: {'kit_label': ['ride'], 'hit_label': ['stick'], 'tech_label': ['bell']},
	augs: {}
	labels: {'kit_label': ['ride'], 'hit_label': ['stick'], 'tech_label': ['normal']},
	augs: {}
	labels: {'kit_label': ['snare'], 'hit_label': ['stick'], 'tech_label': ['normal

In [5]:
# Resample them to a lower sampling rate
resampled = [reduce_sample_rate(data=sample, new_rate=data_info["down_sampling_rate"], verbose=True) for sample in samples]
save_resamples = [reduce_sample_rate(data=sample, new_rate=data_info["save_sampling_rate"], verbose=True) for sample in samples]

Resampling 21.845s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 0.71s)
Resampling 21.845s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 0.33s)
Resampling 21.845s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 0.28s)
Resampling 21.845s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 0.3s)
Resampling 43.691s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 1.38s)
Resampling 21.845s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 0.34s)
Resampling 21.845s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 0.29s)
Resampling 21.845s of data: from 384000 samples/s to ~3000 samples/s
	Done to 3000.0 samples/s! (Execution time: 0.3s)
Resampling 43.691s of data: from 384000 sa

In [9]:
# Free up samples data
del samples
gc.collect()

[]

In [12]:
# Get the onsets of each of the hits in the original samples, by analysing the resampled audio.
onsets = []
for i in range(no_of_samples):
    s = save_resamples[i]
    rs = resampled[i]
    rs_ebn = energy_based_novelty(rs.data, data_info["hanning_window_size"], verbose=True)
    rs_threshold = threshold_func(rs_ebn, int(data_info["down_sampling_rate"]/5), 2*stats.stdev(rs_ebn.tolist()), 5, verbose=True)
    rs_onsets = onset_detection(rs_ebn, rs_threshold, data_info["onset_trim"], verbose=True)
    current_onsets = rs_onsets*int(s.rate/rs.rate)
    onsets.append(current_onsets)

Calculating ebn of samples...
	Done. (Execution time: 20.23s)
Creating a threshold function for enb function...
	Done. (Execution time: 3.001298427581787s)
Finding onsets (peak peaking the detection function)...
	26 onsets found. (Execution time: 0.03s)
Calculating ebn of samples...
	Done. (Execution time: 19.9s)
Creating a threshold function for enb function...
	Done. (Execution time: 2.9463202953338623s)
Finding onsets (peak peaking the detection function)...
	10 onsets found. (Execution time: 0.03s)
Calculating ebn of samples...
	Done. (Execution time: 13.53s)
Creating a threshold function for enb function...
	Done. (Execution time: 1.9328875541687012s)
Finding onsets (peak peaking the detection function)...
	17 onsets found. (Execution time: 0.02s)
Calculating ebn of samples...
	Done. (Execution time: 16.83s)
Creating a threshold function for enb function...
	Done. (Execution time: 2.4296045303344727s)
Finding onsets (peak peaking the detection function)...
	47 onsets found. (Execu

In [13]:
# Using the onsets crop a length of audio from the original which captures a hit, save this crop.
cropped_samples = []
for i in range(no_of_samples):
    s = save_resamples[i]
    current_onsets = onsets[i]
    for onset in current_onsets:
        # Make a crop in the np array of samples
        start = onset
        end = int(start+data_info["length"]*s.rate)
        cropped_samples.append(Sample_data(labels=s.labels, augmentations=s.augmentations, rate=s.rate, data=np.float32(s.data[start:end])))

In [19]:
gc.get_stats()

[{'collected': 1602, 'collections': 232, 'uncollectable': 0},
 {'collected': 662, 'collections': 21, 'uncollectable': 0},
 {'collected': 107, 'collections': 4, 'uncollectable': 0}]

In [16]:
onsets = None
resampled = None
save_resamples = None
gc.collect()

0

In [17]:
gc.garbage

[]

In [23]:
len(gc.get_referents(resampled))

0

In [24]:
save_data(data_type="cropped", data_set=cropped_samples, verbose=True)

Attempting to save cropped data and labels...
	Saved: 216/216 files.


True

In [27]:
cropped_samples = None
gc.collect()

0

In [30]:
gc.get_threshold()

(700, 10, 10)

In [28]:
gc.get_objects()

[{'contexts': ((), None)},
 <weakref at 0x00000288806FFB88; dead>,
 [],
 [],
 [],
 [(SelectorKey(fileobj=380, fd=380, events=1, data=(<Handle BaseAsyncIOLoop._handle_events(380, 1)>, None)),
   1)],
 [],
 ({380, 1120, 1128}, set(), set(), None),
 {'contexts': ((), None)},
 <cell at 0x00000288F25000A8: Runner object at 0x00000288F2412A90>,
 <Future pending cb=[_make_coroutine_wrapper.<locals>.wrapper.<locals>.<lambda>() at c:\users\danzi\appdata\local\programs\python\python35\lib\site-packages\tornado\gen.py:347, IOLoop.add_future.<locals>.<lambda>() at c:\users\danzi\appdata\local\programs\python\python35\lib\site-packages\tornado\ioloop.py:719]>,
 {'_callbacks': [<function tornado.gen._make_coroutine_wrapper.<locals>.wrapper.<locals>.<lambda>(_)>,
   <function tornado.ioloop.IOLoop.add_future.<locals>.<lambda>(future)>],
  '_loop': <_WindowsSelectorEventLoop running=True closed=False debug=False>},
 [<function tornado.gen._make_coroutine_wrapper.<locals>.wrapper.<locals>.<lambda>(_)>,

In [29]:
gc.isenabled()

True

In [31]:
del onsets
del resampled
del save_resamples
del cropped_samples
gc.collect()

235

In [32]:
gc.collect()

0